# Finding the Right Spot

Requirements:
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Account managers need to travel a lot.
- Everyone in the company is between 25 and 40, give them some place to go party.

In [2]:
#importing necessary libraries
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv
from pymongo import MongoClient
import geopandas as gpd
from functools import reduce
import operator
import geopandas as gpd

In [3]:
#functions
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

# Calculate Distance Between GPS Points

import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [48]:
#checking connection and getting the database 'companies'
conn = MongoClient("localhost:27017")
db = conn.get_database("ironhack")
collection = db.get_collection("companies")

In [5]:
c = collection

I filtered the database 'companies' to find those with offices in Dublin and which had raised over €1.0M. The dataframe below shows the results obtained.

### Why Dublin?
- **Low Taxes**
Thanks to the governmental effort to welcome new businesses and support creation of a more diverse job market, Dublin has the most attractive corporate tax rates in the world – just 12,5 %. R&D expenditure generates a 25% tax credit for offset against corporate taxes in addition to a tax deduction at 12.5% — this means that companies can claim a refund of €37.50 for every €100 worth of R&D expenditure.
- **Talented workforce**
Significantly cheaper than London, Dublin attracts young tech talent from all over the world. Dublin produces over 26 thousand graduates every year and is home to three internationally-ranked universities — Trinity College Dublin, University College Dublin and Dublin City University. That’s already a huge talent pool right there. Global Talent Competitiveness Index ranked Dublin among the top 10 cities for attracting and retaining talent.
- **Ireland Help**
Ireland’s Local Enterprise Office (LEO) spearheads business support for companies looking to relocate to Ireland through mentoring, training and grants ranging from €15,000 to help companies research market demand to €80,000 for expansion.

In [6]:
pd.DataFrame(c.find(
    {"$and": [
        {"offices.city": 'Dublin'},
        {"total_money_raised": {'$gt': '1.0'}}
    ]},
    {"name": 1, "rating": 1, "total_money_raised": 1, 'offices.latitude':1, 'offices.longitude':1}
))

,_id,name,total_money_raised,offices
0,52cdef7c4bab8bd6752986b7,eRepublik,€2.75M,"[{'latitude': 40.43094, 'longitude': -3.695289..."
1,52cdef7d4bab8bd675298963,Iceberg,€400k,"[{'latitude': 53.344104, 'longitude': -6.267494}]"
2,52cdef7d4bab8bd675298aec,Dial2Do,€1M,"[{'latitude': 53.34202, 'longitude': -6.250684}]"
3,52cdef7d4bab8bd675299e60,payByMobile,€750k,"[{'latitude': None, 'longitude': None}]"
4,52cdef7e4bab8bd67529b0e2,DediServe,€1M,"[{'latitude': None, 'longitude': None}, {'lati..."
5,52cdef7e4bab8bd67529b59e,Branded Payment Solutions,€100k,"[{'latitude': 53.344104, 'longitude': -6.26749..."
6,52cdef7e4bab8bd67529b913,Xiam,€1M,"[{'latitude': None, 'longitude': None}]"
7,52cdef7f4bab8bd67529c55b,Clavis Technology,€3.66M,"[{'latitude': None, 'longitude': None}, {'lati..."


I decided to focus on Clavis Technology location after having analyzed Dial2Do and Iceberg mainly because of the total money raised and given the 'products/services' offered by the company. By looking at the information available, I saw Clavis Tech as a successfull tech startup company - At this point, I thought about two options: 1) buy Clavis Tech offices for my new gaming company or 2) buy an office building close to that location.
Clavis Tech provides software products and services. The Company offers cloud-based electronic commerce intelligence and data quality performance management solutions. 

Now, let's see if the Clavis Tech office location matches my chosen requirements, which include:
- A starbucks coffee since it is very popular among employees
- Some place to party since workforce is aged between 25 and 40
- A good transportation network since they need to have easy access to the airport

In [7]:
dublin = {'type': 'Point', 'coordinates': [-6.25415, 53.32306]}
clavis_tech = {'type': 'Point', 'coordinates': [-6.258292, 53.346976]}

In [49]:
#map_dublin
map_dublin = folium.Map(location = [53.346976,-6.258292], zoom_start = 15)

In [50]:
#Adding Clavis Technologies (searched coordinates in google maps since the data was missing from companies' database)
#Location Clavis
clavis_lat =  53.346976
clavis_long = -6.258292

#creating the icon for Clavis
icono = Icon(color = "red",
             prefix = "fa",
             icon = "briefcase",
             icon_color = "black",
             tooltip = "Clavis Tech"
)
loc = [clavis_lat, clavis_long]
#saving the marker in one variable
marker_clavis = Marker(location = loc, icon = icono)
#let's add the marker to the original dublin map where we already had Dial2do location saved
marker_clavis.add_to(map_dublin)
map_dublin

As we can see above, the red suitcase marker represents Clavis Technology, the starting point that I have used to conduct my ideal office location based on the above mentioned requirements.

By using Foursquare API [https://foursquare.com/developers/apps], I was able to conduct different calls to find the above mentioned requirements. Additionally, I also studied other venues such as veterinarian, vegan restaurants etc, which can be found in the notebook called 'Analysing Dublin Office Location'. However, I decided not to include them since the results were not accurate.
I used Clavis Tech location coordinates as the starting point and analyzed the different venues from there.

## Starbucks

Let's take a look at the starbucks requirement. Further details can be found in 'Analysing Dublin Office Location' notebook. 

In [30]:
load_dotenv()

True

In [31]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [51]:
#url and Starbucks venue chain ID
enlace = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [52]:
#params to be taken into account in calling FourSquare API
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": f"{starbucks}",
          "limit" : 10
}

In [53]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [54]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
starbucks = decoded.get('items')

In [55]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [56]:
new_list = []
for diccionario in starbucks:
    starbuckslist = {}
    starbuckslist["name"] = getFromDict(diccionario,mapa_nombre)
    starbuckslist["latitud"] = getFromDict(diccionario,latitud)
    starbuckslist["longitud"] = getFromDict(diccionario,longitud)
    new_list.append(starbuckslist)

In [57]:
dataframestarbucks = pd.DataFrame(new_list)
dataframestarbucks.head()

,name,latitud,longitud
0,DoubleTree by Hilton,53.346559,-6.265901
1,Farrier & Draper,53.342417,-6.262255
2,"The Shelbourne Dublin, A Renaissance Hotel",53.338972,-6.256014
3,TOSS'D Noodles & Salads,53.348874,-6.247886
4,FXB Steak & Seafood,53.337314,-6.251954


In [58]:
#Let's see which starbucks is closer to Clavis Tech
clavis_tech_coord = 53.346976,  -6.258292
starbucks = {
    'DoubleTree by Hilton': (53.346559,  -6.265901),
    'Farrier & Draper': (53.342417,  -6.262255),
    'The Shelbourne Dublin': (53.338972, -6.247886),
    'TOSSD Noodles & Salads': (53.348874,  -6.247886),
    'FXB Steak & Seafood': (53.337314,  -6.251954)
}

for places, coord in starbucks.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see DoubleTree Hilton is 500m away -> chosen starbucks

DoubleTree by Hilton 507.3533169653642
Farrier & Draper 571.2965056892294
The Shelbourne Dublin 1126.9636091398845
TOSSD Noodles & Salads 722.4592739043518
FXB Steak & Seafood 1154.1466690405266


In [59]:
#Adding closest Starbucks to dublin map
#Location Clavis
starbucks_lat =  53.346559
starbucks_long = -6.265901

#creating the icon for Starbucks
icono = Icon(color = "darkgreen",
             prefix = "fa",
             icon = "coffee",
             icon_color = "black",
             tooltip = "Starbucks"
)
loc = [starbucks_lat, starbucks_long]
#saving the marker in one variable
marker_starbucks = Marker(location = loc, icon = icono)
#let's add the marker to the map where we already had Clavis Tech location saved
marker_starbucks.add_to(map_dublin)
map_dublin

## Irish Pubs

Now, let's take a look at the 'party' requirement. Further details can be found in 'Analysing Dublin Office Location' notebook. 

In [41]:
enlace = 'https://api.foursquare.com/v2/venues/explore'
irish_pubs = "52e81612bcbc57f1066b7a06"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": f"{irish_pubs}",
          "limit" : 10
}

In [42]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [43]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
irish_pubs = decoded.get('items')

In [44]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [45]:
new_list2 = []
for diccionario in irish_pubs:
    irishpubslist = {}
    irishpubslist["name"] = getFromDict(diccionario,mapa_nombre)
    irishpubslist["latitud"] = getFromDict(diccionario,latitud)
    irishpubslist["longitud"] = getFromDict(diccionario,longitud)
    new_list2.append(irishpubslist)
    
dataframeirish = pd.DataFrame(new_list2)
dataframeirish.head()

,name,latitud,longitud
0,Trinity College Old Library & The Book of Kell...,53.343692,-6.256907
1,FXB Grill,53.344593,-6.263811
2,Fallon & Byrne Food Store,53.343074,-6.263233
3,Fallon & Byrne Wine Cellar,53.343078,-6.263275
4,FXB Steak & Seafood,53.337314,-6.251954


In [46]:
#Let's now calculate distantes to irish pubs
clavis_tech_coord = 53.346976,  -6.258292
irish_pubs = {
    'Trinity College': (53.343692,  -6.256907),
    'FXB Grill': (53.344593,  -6.263811),
    'Fallon & Byrne Food Store': (53.343074, -6.263233),
    'Fallon & Byrne Wine Cellar': (53.343078,  -6.263275),
    'FXB Steak & Seafood': (53.337314,  -6.251954)
}

for places, coord in irish_pubs.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see there are many irish pubs nearby: Trinity College and FXB Grill less than 500m away

Trinity College 376.66677437137986
FXB Grill 452.2702595799134
Fallon & Byrne Food Store 544.0621470953885
Fallon & Byrne Wine Cellar 545.3947205415539
FXB Steak & Seafood 1154.1466690405266


In [60]:
#Adding closest Irish pub to dublin map
#Location Irish Pub
pub_lat =  53.343692
pub_long = -6.256907

#creating the icon for Irish Pub
icono = Icon(color = "orange",
             prefix = "fa",
             icon = "beer",
             icon_color = "black",
             tooltip = "Irish Pub"
)
loc = [pub_lat, pub_long]
#saving the marker in one variable
marker_pub = Marker(location = loc, icon = icono)
#let's add the marker to the map where we already had Clavis Tech location saved
marker_pub.add_to(map_dublin)
map_dublin

By taking a look at the map, I can see there is a good transportation network too, which will make the airport access easier and faster since the account managers need to travel a lot. In fact, Westmoreland tram station is only 80m away and there are other close tram stops nearby, such as the one in Tara St and in Trinity Luas.

## Conclusion

Clavis Tech office buildings seem to be the perfect spot for the new gaming company since it meets all the above mentioned requirements.The coordinates are: [-6.258292, 53.346976]. 